In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup


c:\Users\danfi\miniconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
df = pd.read_csv(r"C:\Users\danfi\VSCode\BERT\big.csv",encoding='latin1')

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['Content'] = df['Content'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danfi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danfi\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danfi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [17]:
df.groupby('gender').describe()

FileName                                                   Content  \
          count unique                                   top freq   count   
gender                                                                      
0          4017   4017  000235a2ba2f48231b7d24e1f08d7878.txt    1    4017   
1          5483   5483  0000e06e07496624211632e8e264126c.txt    1    5483   

                                                                       
       unique                                                top freq  
gender                                                                 
0        4017  damn these 2 games kicked my ass today.,huge w...    1  
1        5483  Paranoia is settin'in!,\Sumbuddy's gonna get a...    1

In [18]:
df.shape


(9500, 3)

In [19]:
df['gender'].value_counts()

1    5483
0    4017
Name: gender, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Content'],df['gender'], stratify=df['gender'])

In [21]:
X_train.head(4)

6257    Rain-ed in...,Fujitsu Hard Drives: For those w...
4228    never thought fighting robots can be soooo coo...
2177    Cnt help but feel I'm OVERKILLIN' myself. Sims...
7750    -gives self a mega facepalm-,:DDD Friday the 1...
Name: Content, dtype: object

In [22]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [23]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])


In [24]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [25]:
len(X_train)


7125

In [26]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)


In [27]:
model.fit(X_train, y_train, epochs=10)


Epoch 1/10
223/223 [==============================] - 67s 252ms/step - loss: 0.6794 - accuracy: 0.5808
Epoch 2/10
223/223 [==============================] - 56s 253ms/step - loss: 0.6552 - accuracy: 0.6129
Epoch 3/10
223/223 [==============================] - 55s 245ms/step - loss: 0.6403 - accuracy: 0.6335
Epoch 4/10
223/223 [==============================] - 54s 244ms/step - loss: 0.6323 - accuracy: 0.6415
Epoch 5/10
223/223 [==============================] - 54s 244ms/step - loss: 0.6204 - accuracy: 0.6519
Epoch 6/10
223/223 [==============================] - 56s 252ms/step - loss: 0.6146 - accuracy: 0.6609
Epoch 7/10
223/223 [==============================] - 59s 265ms/step - loss: 0.6055 - accuracy: 0.6700
Epoch 8/10
223/223 [==============================] - 83s 373ms/step - loss: 0.6046 - accuracy: 0.6745
Epoch 9/10
223/223 [==============================] - 81s 365ms/step - loss: 0.5989 - accuracy: 0.6769
Epoch 10/10
223/223 [==============================] - 81s 363ms/step - l

In [30]:
from sklearn.metrics import classification_report
import numpy as np

y_predicted = model.predict(X_test).flatten()

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

print(classification_report(y_test, y_predicted))


75/75 [==============================] - 29s 379ms/step
              precision    recall  f1-score   support

           0       0.73      0.49      0.59      1004
           1       0.70      0.87      0.77      1371

    accuracy                           0.71      2375
   macro avg       0.71      0.68      0.68      2375
weighted avg       0.71      0.71      0.69      2375

